## Imports & Variables

In [16]:
#Imports

import os
import time
import joblib
from tqdm import tqdm
from IPython.display import clear_output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import _california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import *
from sklearn.ensemble import *

from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

#Settings
plt.style.use('dark_background')

#Colonnes du dataset
"""
- MedInc        median income in block group
- HouseAge      median house age in block group
- AveRooms      average number of rooms per household
- AveBedrms     average number of bedrooms per household
- Population    block group population
- AveOccup      average number of household members
- Latitude      block group latitude
- Longitude     block group longitude
- MedHouseVal   median house value
"""

#Creation du dataframe avec le dataset
dataset = _california_housing.fetch_california_housing(as_frame=True)
df = dataset.frame

In [2]:
#Variables

#Definition de la target
target_name = "MedHouseVal"
target = df[target_name]

#Colonnes exclues pour le test
columns_to_drop = [
    target_name, 
    # "Population", 
    # "AveOccup", 
    # "AveBedrms", 
    # "HouseAge", 
    # "AveRooms"
]
data = df.drop(columns=columns_to_drop)

#Modeles a tester

linear_models = [
    LinearRegression(),
    Ridge(),
    RidgeCV(),
    SGDRegressor(),
    Lasso(),
]

models = [
    AdaBoostRegressor(),  
    BaggingRegressor(n_jobs=-1), 
    ExtraTreesRegressor(n_jobs=-1), 
    GradientBoostingRegressor(), 
    RandomForestRegressor(n_jobs=-1), 
    HistGradientBoostingRegressor()
]

boost_and_bag_models = [
    #AdaBoostRegressor(base_estimator=BaggingRegressor(n_jobs=-1)),
    #AdaBoostRegressor(base_estimator=ExtraTreesRegressor(n_jobs=-1)),
    #AdaBoostRegressor(base_estimator=GradientBoostingRegressor()),
    #AdaBoostRegressor(base_estimator=RandomForestRegressor(n_jobs=-1)),
    AdaBoostRegressor(random_state=69, learning_rate=1,n_estimators=115, base_estimator=HistGradientBoostingRegressor(learning_rate=0.1, max_iter=1000, max_depth=7, warm_start=True, max_leaf_nodes=None, random_state=69,  max_bins=255)),

    #BaggingRegressor(base_estimator=BaggingRegressor(n_jobs=-1)),
    #BaggingRegressor(base_estimator=ExtraTreesRegressor(n_jobs=-1)),
    #BaggingRegressor(base_estimator=GradientBoostingRegressor()),
    #BaggingRegressor(base_estimator=RandomForestRegressor(n_jobs=-1)),
    #BaggingRegressor(base_estimator=HistGradientBoostingRegressor()),
]

#Nombre de trainings par modele
attemps = 1

## 1 - Début de l'analyse des données

In [ ]:
#Infos du dataset

print(dataset.DESCR)

In [ ]:
#Heatmap

data = df.drop(columns=["Latitude", "Longitude"])
plt.figure(figsize=(12,7))
sns.heatmap(data.corr(),cbar=True,annot=True,cmap='Blues')

In [ ]:
#Prix moyen en fonction de l'age des maisons

sns.lineplot(df,
                x="HouseAge", y="MedHouseVal", alpha=0.5)
plt.title("MedHouseVal based on HouseAge")

plt.show()

In [ ]:
#Prix moyen en fonction du nombre de pièces

sns.lineplot(df,
                x="AveBedrms", y="MedHouseVal", alpha=0.5)
plt.title("MedHouseVal based on AveRooms")

plt.show()

In [ ]:
#Prix moyen en fonction de la localisation géographique

sns.scatterplot(df,
                x="Longitude", y="Latitude",
                size="MedHouseVal", hue="MedHouseVal",
                palette="viridis", alpha=0.5)
plt.legend(title="MedHouseVal", bbox_to_anchor=(1.05, 1),
           loc="upper left")
plt.title("Median house value depending of\n their world location")

plt.show()

## 2 - Recherche du modèle adéquat

In [ ]:
#RandomForestClassifier

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.01, random_state=42)

modele_rf = RandomForestClassifier(
    # il s’agit du nombre d’arbres dans la forêt
    n_estimators=5,
    # il s’agit du critère utilisé pour construire les arbres et séparer les branches des arbres
    criterion='gini',
    # il s’agit de la profondeur maximale des arbres utilisés (le nombre de niveaux dans l’arbre de décision)
    max_depth=None,
    # il s’agit du nombre d’échantillons minimal dans une feuille pour refaire une séparation
    min_samples_split=2,
    # il s’agit du nombre d’échantillons minimal pour créer une feuille
    min_samples_leaf=1,
    # il s’agit de la fraction du nombre total d’échantillon minimal pour créer une feuille
    min_weight_fraction_leaf=0.0,
    # il s’agit du nombre maximal de feuilles
    max_leaf_nodes=None,
    # il s’agit de la baisse minimale du critère d’impureté pour faire une séparation
    min_impurity_decrease=0.0,
    # paramètre pour utiliser du bootstrap, si il est à False, le même échantillon est pris pour chaque arbre
    bootstrap=True,
    # ??
    oob_score=False,
    # nombre de traitements à effectuer en parallèle
    n_jobs=None,
    # graine aléatoire
    random_state=None,
    # ??
    verbose=0,
    # ceci permet de repartir du résultat du dernier apprentissage pour faire l’apprentissage
    warm_start=False,
    # il s’agit des poids associés à chaque classe si cela a un sens
    class_weight=None,
    # ??
    ccp_alpha=0.0,
    # si vous voulez réduire le nombre d’observations dans vos échantillons bootstrap
    max_samples=None,
)

modele_rf.fit(x_train, y_train)

In [ ]:
#KNeighborsRegressor

#Colonnes exclues pour le train
data = df.drop(columns=[target_name, "AveOccup", "Population", "HouseAge", "AveBedrms" , "AveRooms"])

#Entrainement
model = KNeighborsRegressor()
model.fit(data, target)

target_predicted = model.predict(data)
print(target[:5])
print(target_predicted[:5])
print(f"Number of correct prediction: "
      f"{(target[:5] == target_predicted[:5]).sum()} / 5")